# Log 01-07-2023

## New workflow
- you do not even need a Model class for temporarily storing anything
- you can create a single model and then duplicate it for model statement; and then deactivate constraints
    - by thinking about how gams works itself
- and create subclasses for definitions, and add extra code for creating sets, params, etc in Pyomo
- or **DIRECTLY write string**

## TODOs
- [ ] generate code from the transformer
    - [x] title
    - [x] option
    - [x] set
    - [x] alias
    - [x] scalar
    - [x] parameter
    - [ ] assignment
    - [ ] if statement
    - [ ] loop
    - [ ] variables
    - [ ] constraints
    - [ ] objective
    - [ ] set bounds?
- [ ] update grammar
    - [ ] un-ignore comments
- [ ] rename `model.py` to `components.py`

## Observations

### 16:03

- grammar: `_ALIAS 		_LPAR index_list _RPAR 	_END -> alias_list`
- transformer for `alias_list`: only index_list is returned, its symbol_type is marked as `alias`
- the `assemble` method should be with index_list

### 16:11

scalar parsed wrong: 0.3271 parsed as 0 and 3271

### 17:00

probably we still need to count something...

### 17:32

alternative structure:
- `Symbol`: take a tree as argument, then kinda parse it partially

### 17:59 The transformer structure is a shit show

### 21:17

The usually returned list is caused by the following structure:
- `symbol 			: symbol_name[suffix][symbol_index]`
- `symbol_name		: identifier`
- `identifier 		: WORD_IDENTIFIER | SIGNED_NUMBER`
- `identifier` is transformed, but `symbol_name` is not (at the time)
- Therefore, when transforming `symbol`, `symbol_name` is its first child as a Tree (not transformed)
    - the children of `symbol_name` is ALWAYS a list
    - but the nested structure caused the confusion

### 22:03

another difference between rule and terminals:
- when `suffix` was assigned to specific strings, those strings were not returned (info lost)
- avoided when it is updated to `_DOT string`
- or you can do `suffix: .lp -> suffix_lp`

### 22:53
Sometimes the tree can be left there... when transformed only need to care about its `.data`

### (01-08) 16:19 transition to small cases

## Code

### check every statement

In [ ]:
from gams2pyomo import GamsParser

f = './test/gams_real_problem/tanksize2.gms'

with open(f, 'r') as in_file:
    gp = GamsParser(in_file)

parse_tree = gp.parse()
res = gp.transform()
print(res)

### start from scratch

In [1]:
from gams2pyomo import GamsParser

f = './test/gams_real_problem/tanksize2.gms'
# f = './test/gams_basic/flow_control/if-elseif-else.gms'
# f = './test/gams_basic/flow_control/if-else.gms'

with open(f, 'r') as in_file:
    gp = GamsParser(in_file)

parse_tree = gp.parse()
# print(parse_tree.pretty())
res = gp.transform()
# print(res)

In [3]:
with open('tmp/test.py', 'w') as file:
    file.write(res)

In [ ]:
x = parse_tree.children[0]
x.children[0]

### test Pyomo grammar

In [14]:
from pyomo.environ import *

m = ConcreteModel()
m.p = Param(initialize=1)
m.pp = Param(mutable=True)

m.pp = m.p._data

In [16]:
# m.pp.pprint()
m.pp.pprint()

pp : Size=1, Index=None, Domain=Any, Default=None, Mutable=True
    Key  : Value
    None : {None: p}
